# Benchmarking Our Algorithm against others

In [7]:
import sys
sys.path.append('../')
from panav.environment.env import WareHouse
from panav.hybrid import WareHouseHG
from panav.util import flowtime,makespan
from panav.PBS.PBS import PBS
from panav.TrafficAwarePlanning import TAHP
from panav.sequential import sequential_planning
from panav.PIBT import PIBT_plan
from panav.SAMP.solvers import Tube_Planning



from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:

bloating_r = 0.5
vmax = 1.0

#### 100 agent configuration ####
n_agents = list(range(10,101,10))
bloating_r = 0.5
vmax = 1.0


n_col = 10
n_row = 10
limits = [(-50,50),(-50,50)] 
shelf_region_x_limit=[-30,30]
shelf_region_y_limit=[-30,30]
corner_padding_x = bloating_r * 10
corner_padding_y = bloating_r * 10
obs_x_margin = 8 * bloating_r
obs_y_margin = 8 * bloating_r

TIMEOUT = 100 # timeout

# Algorithms
algs = {  
        "PIBT": lambda hg: PIBT_plan(hg,vmax,bloating_r,TIMEOUT),
        "S2M2": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Simple_MILP_Planning",TIMEOUT=TIMEOUT)[0],
        "PBS_Tube": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Tube_Planning",TIMEOUT=TIMEOUT)[0],
        "TAHP": lambda hg: TAHP(hg,vmax=vmax,bloating_r=bloating_r,TIMEOUT=TIMEOUT),
        "Seq_Tube": lambda hg: sequential_planning(Tube_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        # "Seq_Temp_Obs": lambda hg: sequential_planning(Simple_MILP_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT)
        }

timeouted = {alg:False for alg in algs}

results = []

env_name = "Warehouse"


In [12]:
# Experiments
cur_N = 0
for i in range(len(n_agents)):
    
    N = n_agents[i]
   

    env = WareHouse(limits = limits, N_agent = N, 
                    shelf_region_x_limit=shelf_region_x_limit,shelf_region_y_limit=shelf_region_y_limit,
                    obs_x_margin = obs_x_margin,
                    obs_y_margin = obs_y_margin,
                    n_col = n_col, n_row = n_row,
                    corner_padding_x=corner_padding_x,corner_padding_y=corner_padding_y)

    print('Constructing environment')
    HG = WareHouseHG(env,bloating_r)

    if N<cur_N:
        continue
  
    # Save checkpoint data
    with open("./data/{}_misc.pkl".format(env_name),'wb') as fp:
        pkl.dump({"TIMEOUT":TIMEOUT,'cur_N':N},fp)
        
    for name,alg in algs.items():

        if timeouted[name]:
            continue
        
        print("Alg:", name, "N agent:",N)

        result = {}
        
        t0 = time()
        plan = alg(HG)
        t = time()-t0
        
        result['plan'] = plan
        result['time'] = t

        result['N_agent'] = N
        result['alg_name'] = name

        result["TIMEOUT"]=TIMEOUT
        result["HG"]=HG

        results.append(result)

        
        if t>TIMEOUT:
            timeouted[name] = True
            print("Time out for algorithm", name, "N agent = ", N)

    # Save checkpoint data
    with open("./data/{}.pkl".format(env_name),'wb') as fp:
        pkl.dump(results,fp)


Constructing environment


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


Alg: PIBT N agent: 10
Alg: S2M2 N agent: 10


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


PBS Time Elapsed: 4.657599925994873
PBS Time Elapsed: 5.124459981918335
PBS Time Elapsed: 5.578452825546265
PBS Time Elapsed: 6.389743089675903
PBS Time Elapsed: 7.432911157608032
PBS Time Elapsed: 7.9047698974609375


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


PBS Time Elapsed: 336.7540090084076
Time out for algorithm S2M2 N agent =  10
Alg: PBS_Tube N agent: 10
PBS Time Elapsed: 4.832066059112549
PBS Time Elapsed: 5.552655220031738
PBS Time Elapsed: 6.505173921585083
PBS Time Elapsed: 7.4456870555877686
PBS Time Elapsed: 8.136440992355347
PBS Time Elapsed: 9.335241079330444
PBS Time Elapsed: 10.300524950027466
PBS Time Elapsed: 11.485928297042847
PBS Time Elapsed: 12.437334060668945
PBS Time Elapsed: 13.407219886779785
PBS Time Elapsed: 14.067738056182861
PBS Time Elapsed: 14.764883995056152
PBS Time Elapsed: 15.436141967773438
PBS Time Elapsed: 16.115083932876587
PBS Time Elapsed: 17.041341304779053
PBS Time Elapsed: 17.778748273849487
PBS Time Elapsed: 18.473863124847412
PBS Time Elapsed: 19.752424955368042
PBS Time Elapsed: 20.850261926651
PBS Time Elapsed: 21.566160202026367
PBS Time Elapsed: 23.057717084884644
PBS Time Elapsed: 24.028027057647705
PBS Time Elapsed: 25.385117292404175
PBS Time Elapsed: 26.4431049823761
PBS Time Elapsed: 